In [1]:
%load_ext autoreload
%autoreload 2


In [2]:
from models import networks
import torch
from torchvision import transforms

In [3]:
model = networks.PerceptualLoss().cuda()

In [10]:
x = (torch.rand(16, 3, 256, 256).cuda() - 0.5) * 2
y = (torch.rand(16, 3, 256, 256).cuda() - 0.5) * 2
neg = (torch.rand(16, 3, 256, 256).cuda() - 0.5) * 2

In [29]:
with torch.no_grad():
    x = (torch.rand(16, 3, 256, 256).cuda() - 0.5) * 2
    y = (torch.rand(16, 3, 256, 256).cuda() - 0.5) * 2
    neg = (torch.rand(16, 3, 256, 256).cuda() - 0.5) * 2
    print(model(x, y, neg))

tensor(55.1514, device='cuda:0')


In [66]:
input_mean = (0.5, 0.5, 0.5)
input_std = (0.5, 0.5, 0.5)
imagenet_mean = (0.485, 0.456, 0.406)
imagenet_std = (0.229, 0.224, 0.225)

In [55]:
class Normalize:
    def __init__(self, mean, std, inplace=False, dtype=torch.float, device='cpu'):
        self.mean = torch.as_tensor(mean, dtype=dtype, device=device)[None, :, None, None]
        self.std = torch.as_tensor(std, dtype=dtype, device=device)[None, :, None, None]
        self.inplace = inplace
        
    def __call__(self, tensor):
        if not self.inplace:
            tensor = tensor.clone()

        tensor.sub_(self.mean).div_(self.std)
        return tensor

In [57]:
tr = transforms.Compose([
    Normalize(mean=[-mean/std for mean, std in zip(input_mean, input_std)], std=[1/std for std in input_std], dtype = x.dtype, device = 'cuda'),
    Normalize(mean=imagenet_mean, std=imagenet_std, dtype = x.dtype, device = 'cuda')
])

In [63]:
y = tr(x)

In [64]:
y.size()

torch.Size([16, 3, 256, 256])